In [1]:
import os
import sys

In [2]:
ROOT_DIR = "../../"
sys.path.insert(0, os.path.abspath(ROOT_DIR + "/src/scraping"))
from scraping import steam_scraper

In [3]:
import requests
import bs4
import pandas as pd
import numpy as np

In [4]:
sc = steam_scraper(ROOT_DIR)

In [5]:
sc.scrape_data()

In [6]:
sc.games

,name,href
0,Rust,https://store.steampowered.com/app/252490/Rust/
1,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...
2,Apex Legends™,https://store.steampowered.com/app/1172470/Ape...
3,Destiny 2,https://store.steampowered.com/app/1085660/Des...
4,Phasmophobia,https://store.steampowered.com/app/739630/Phas...
...,...,...
89963,PowerWash Simulator,https://store.steampowered.com/app/1290000/Pow...
89964,Hidden Shelter,https://store.steampowered.com/app/1422820/Hid...
89965,Realm Engine | Virtual Tabletop,https://store.steampowered.com/app/1427700/Rea...
89966,Go Gold Castle,https://store.steampowered.com/app/1443700/Go_...


***

In [7]:
def collect_sys_reqs(chunksize=2000, starting_chunk=None, max_chunk=None):
    default_max_chunk = int(np.ceil(sc.games.shape[0] / chunksize))
    if not(new_max_chunk is None):
        assert isinstance(new_max_chunk, int), "Max chunk must be an integer"
        assert new_max_chunk >= 0, "Only a non-negative max chunk is valid"
        assert new_max_chunk <= default_max_chunk, "Max chunk cannot go beyond the current max of {}".format(default_max_chunk)
        max_chunk = new_max_chunk
    else:
        max_chunk = default_max_chunk
        
    if starting_chunk is None:
        starting_chunk = 0
    else:
        assert isinstance(starting_chunk, int), "Starting chunk must be an integer"
        assert starting_chunk >= 0, "Only non-negative starting chunk are valid"
        assert starting_chunk <= max_chunk, "Max chunk given current settings is only {}. Starting chunk cannot be bigger than the max chunk.".format(max_chunk)
        
    fname_root = ROOT_DIR + "data/temp/"
    if not os.path.exists(fname_root):
        os.mkdir(fname_root)
    for chunk in range(starting_chunk, max_chunk):
        fname = fname_root + str(chunk)
        
        start = chunk * chunksize
        end = (chunk * chunksize) + chunksize
        if end > sc.games.shape[0]:
            end = sc.games.shape[0] + 1
        print("Starting chunk {}:{}-{}".format(chunk, start, end))
        game_infos = []
        # Begin fetching all data for a chunk
        for ix in range(start, end):
            print("\tProcessing game #{}".format(ix), end='\r')
            target_game = sc.games.iloc[ix]
            game_info = sc.get_game_info(target_game.href)
            for key in game_info:
                target_game[key] = game_info[key]
            game_infos.append(target_game)
        try:
            # Save chunk data to a temp file
            game_infos = pd.DataFrame(game_infos).to_csv(fname_root + f"game_sys_reqs_{chunk}.csv", index=False)
        except:
            # If unable to write the data, save it
            self.temp = game_infos
            raise Exception("Unable to save file. Temporary data saved to self.temp attribute")

In [11]:
%time collect_sys_reqs(max_chunk = 1)

Starting chunk 0:0-2000
Wall time: 15min 41s99


In [ ]:
import requests
import bs4

In [ ]:
link = "https://store.steampowered.com/app/1059530/Valve_Index_Headset/?snr=1_7_7_230_150_3"
page = requests.get(link)
content = bs4.BeautifulSoup(page.content)

In [ ]:
content.find("div", class_="details_block").text.split(':')[-1].strip()